In [ ]:
!pip install -qU \
  pinecone-client==3.1.0 \
  pinecone-datasets==0.7.0 \
  pinecone-notebooks==0.1.1

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-genai 1.8.0 requires pydantic<3.0.0,>=2.0.0, but you have pydantic 1.10.21 which is incompatible.
langchain-core 0.3.49 requires pydantic<3.0.0,>=2.5.2; python_full_version < "3.12.4", but you have pydantic 1.10.21 which is incompatible.
albumentations 2.0.5 requires pydantic>=2.9.2, but you have pydantic 1.10.21 which is incompatible.
langchain 0.3.21 requires pydantic<3.0.0,>=2.7.4, but you have pydantic 1.10.21 which is incompatible.
wandb 0.19.8 requires pydantic<3,>=2.6, but you have pydantic 1.10.21 which is incompatible.


In [ ]:
!pip install --upgrade pydantic -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pinecone-datasets 0.7.0 requires pydantic<2.0.0,>=1.10.5, but you have pydantic 2.11.1 which is incompatible.


In [ ]:
!pip install -U bitsandbytes -q

In [ ]:
from bs4 import BeautifulSoup
import requests
import time

def scrape_trustpilot_reviews(company_name, num_pages=5, delay=2):
    """
    Scrapes reviews from Trustpilot for a given company.

    Args:
        company_name (str): The name of the company to scrape reviews for (e.g., 'aetna').
        num_pages (int, optional): The number of pages to scrape. Defaults to 5.
        delay (int, optional): The delay in seconds between page requests. Defaults to 2.

    Returns:
        list: A list of review strings, or an empty list if an error occurred.
    """
    trustpilot_url = f'https://www.trustpilot.com/review/www.{company_name}.com?page='
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }

    def scrape_page(url):
        """Scrape reviews from a single page."""
        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            reviews = soup.find_all('p', attrs={'data-service-review-text-typography': 'true'})
            return [review.get_text() for review in reviews]
        else:
            print(f"No more webpages left to scrape. Status code: {response.status_code}")
            return []

    # Try to scrape first to know if the company exists
    url = trustpilot_url + str(1)
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
      print(f"Company '{company_name}' not found on Trustpilot.")
      return []

    all_reviews = []
    for page in range(1, num_pages + 1):
        url = trustpilot_url + str(page)
        print(f"Scraping Trustpilot page {page}...")
        reviews = scrape_page(url)
        if not reviews:
            break  # Stop if no reviews are found on a page
        all_reviews.extend(reviews)
        time.sleep(delay)

    return all_reviews

In [ ]:
def scrape_bestcompany_reviews(company_name, num_pages=5, delay=2):
    """
    Scrapes reviews from Best Company for a given company.

    Args:
        company_name (str): The name of the company to scrape reviews for (e.g., 'aetna').
        num_pages (int, optional): The number of pages to scrape. Defaults to 5.
        delay (int, optional): The delay in seconds between page requests. Defaults to 2.

    Returns:
        list: A list of review strings, or an empty list if an error occurred.
    """
    bestcompany_url = f'https://bestcompany.com/health-insurance/{company_name}?page='
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }

    def scrape_page(url):
        """Scrape reviews from a single page."""
        response = requests.get(url, headers=headers)

        if response.status_code == 200:
          soup = BeautifulSoup(response.content, 'html.parser')
          reviews = soup.find_all('div', class_='whitespace-pre-line break-words')
          return [review.get_text() for review in reviews]
        else:
          print(f"No more webpages left to scrape. Status code: {response.status_code}")
          return []

    # Try to scrape first to know if the company exists
    url = bestcompany_url + str(1)
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
      print(f"Company '{company_name}' not found on Best Company.")
      return []

    all_reviews = []
    for page in range(1, num_pages + 1):
        url = bestcompany_url + str(page)
        print(f"Scraping Best Company page {page}...")
        reviews = scrape_page(url)
        if not reviews:
            break  # Stop if no reviews are found on a page
        all_reviews.extend(reviews)
        time.sleep(delay)

    return all_reviews

In [ ]:
# Get the company name from the user
company_name = input("Enter the company name (e.g., aetna): ")

Enter the company name (e.g., aetna): aetna


In [ ]:
# Scrape reviews from both Trustpilot and Best Company
trustpilot_reviews = scrape_trustpilot_reviews(company_name, num_pages=3)
bestcompany_reviews = scrape_bestcompany_reviews(company_name, num_pages=3)

Scraping Trustpilot page 1...
Scraping Trustpilot page 2...
Scraping Trustpilot page 3...
Scraping Best Company page 1...
Scraping Best Company page 2...
Scraping Best Company page 3...


In [ ]:
# Combine reviews into a single list
all_reviews = trustpilot_reviews + bestcompany_reviews

In [ ]:
# To see the output, run the code.
if all_reviews:
    print(f"Total Number of Reviews Scraped for {company_name}: {len(all_reviews)}")
else:
    print("No reviews scraped from either site.")

Total Number of Reviews Scraped for aetna: 208


In [ ]:
import pinecone
import os
from pinecone import Pinecone, ServerlessSpec
import time

pine_key = "pcsk_6CjF3M_TVzGxF6Fnzq9mD1eFk33hRRRSrjysqAP4ndVRDNGxNfFkeKw3tq9TtwLz2BhuQT"

pc = Pinecone(
        api_key=pine_key
    )

index_name = 'store-embeddings'

if index_name not in pc.list_indexes().names():
  pc.create_index(
    name=index_name,
    dimension=1024,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

# connect to index
index = pc.Index(index_name)
time.sleep(1)

# view index stats
index.describe_index_stats()

{'dimension': 1024,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [ ]:
from transformers import AutoTokenizer, AutoModel
from langchain.text_splitter import SpacyTextSplitter
import torch
import multiprocessing
from tqdm import tqdm

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
embedding_tokenizer = AutoTokenizer.from_pretrained('jinaai/jina-embeddings-v3')
embedding_model = AutoModel.from_pretrained('jinaai/jina-embeddings-v3', trust_remote_code=True).to(device)


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.11/dist-package

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.11/dist-package

AttributeError: _ARRAY_API not found

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
text_splitter = SpacyTextSplitter(chunk_size=500)

In [ ]:
# Function to generate embeddings
def generate_embeddings(texts):
    embedding_model.eval()
    with torch.no_grad():
        inputs = embedding_tokenizer(texts, padding=True, truncation=True, return_tensors='pt').to(device)
        outputs = embedding_model(**inputs)
        embeddings = outputs.last_hidden_state.mean(dim=1).cpu().float().numpy()
    return embeddings

In [ ]:
# Batch processing
batch_size = 32


for review_id, review in enumerate(all_reviews):
    chunks = text_splitter.split_text(review)
    chunk_embeddings = []
    chunk_ids = []
    chunk_metadata = []

    for chunk_index, chunk in enumerate(chunks):
        chunk_embeddings.append(chunk)
        chunk_ids.append(f"{review_id}_{chunk_index}")
        chunk_metadata.append({"review_id": review_id, "chunk_index": chunk_index, "text": chunk})

        if len(chunk_embeddings) == batch_size:
            embeddings = generate_embeddings(chunk_embeddings)
            vectors = list(zip(chunk_ids, embeddings, chunk_metadata))
            index.upsert(vectors)
            chunk_embeddings, chunk_ids, chunk_metadata = [], [], []

    # Upsert remaining chunks
    if chunk_embeddings:
        embeddings = generate_embeddings(chunk_embeddings)
        vectors = list(zip(chunk_ids, embeddings, chunk_metadata))
        index.upsert(vectors)

print("Upserting completed!")

/usr/local/lib/python3.11/dist-packages/spacy/pipeline/lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)


Upserting completed!


In [ ]:
# Function to query Pinecone
def query_pinecone(query, top_k=3):
    query_embedding = generate_embeddings(query, task='retrieval.query')
    results = index.query(query_embedding, top_k=top_k)
    return results

In [ ]:
query = "How far away is the earth from sun?"

# create the query vector
xq = embedding_model.encode(query).tolist()

# now query
xc = index.query(vector=xq, top_k=3, include_metadata=True)

for result in xc['matches']:
    print(f"{round(result['score'], 2)}: {result['metadata']['text']}")
    print("-------------------------------------------------------------------------")

0.32: This is with an overpriced higher end plan.
-------------------------------------------------------------------------
0.32: I pay $1000 out of pocket and my work pays an extra $800 and I pay 45-55% of the bills and my family limit is $9000 a year before they cover it all?

How is that a good insurance plan?

Some of the 5* comments are extremely general and repetitive.

I expect paid for or bot written.

Read the less than stellar reviews and you get a better picture.
-------------------------------------------------------------------------
0.32: Unacceptable!  

My family and I will be displaced by then as I have a max of 3 days to move out from the premises.

My wife had disability once for maternity leave and the entire process from start to finish was HALF of what I've already been waiting.

Be warned, this company is a joke!
-------------------------------------------------------------------------


In [ ]:
retrieved_text = " ".join([result['metadata']['text'] for result in xc['matches']])
retrieved_text

"This is with an overpriced higher end plan. I pay $1000 out of pocket and my work pays an extra $800 and I pay 45-55% of the bills and my family limit is $9000 a year before they cover it all?\n\nHow is that a good insurance plan?\n\nSome of the 5* comments are extremely general and repetitive.\n\nI expect paid for or bot written.\n\nRead the less than stellar reviews and you get a better picture. Unacceptable!  \n\nMy family and I will be displaced by then as I have a max of 3 days to move out from the premises.\n\nMy wife had disability once for maternity leave and the entire process from start to finish was HALF of what I've already been waiting.\n\nBe warned, this company is a joke!"

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_path = "microsoft/Phi-4-mini-instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_path,
    device_map="cuda",
    load_in_8bit=True,
)

tokenizer = AutoTokenizer.from_pretrained(model_path)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
#index.delete(delete_all=True)

In [ ]:
prompt = f"""### Context:
{retrieved_text}

### Instructions:
You are a helpful assistant answering questions based solely on the provided context.
Respond in the third person without referencing individuals in the context.
If the information needed to answer the question is not in the context, respond with "I don't know".
Do not use any knowledge beyond what is in the context.

### Question:
{query}

### Answer:
"""
#

In [ ]:
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

with torch.no_grad():
  outputs = model.generate(
            **inputs,
            max_new_tokens=200,
            temperature=0.1,
            do_sample=False
        )

    # Extract the response (removing the prompt)
response = tokenizer.decode(outputs[0][inputs.input_ids.shape[1]:], skip_special_tokens=True)

In [ ]:
response

"I don't know."

In [ ]:
from bs4 import BeautifulSoup
import requests
import time

In [ ]:
def scrape_reviews(company_name, sources=['bestcompany', 'trustpilot'], num_pages=5, delay=2):
    """
    Scrapes reviews from multiple sources for a given company and returns them in a single combined list.

    Args:
        company_name (str): The name of the company to scrape reviews for (e.g., 'aetna').
        sources (list, optional): The sources to scrape from. Defaults to ['bestcompany', 'trustpilot'].
        num_pages (int, optional): The number of pages to scrape per source. Defaults to 5.
        delay (int, optional): The delay in seconds between page requests. Defaults to 2.

    Returns:
        list: A combined list of all review strings from all sources.
    """
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }

    all_reviews = []

    # Define source-specific configurations
    source_configs = {
        'bestcompany': {
            'url_template': f'https://bestcompany.com/health-insurance/{company_name}?page=',
            'selector': ('div', {'class_': 'whitespace-pre-line break-words'}),
        },
        'trustpilot': {
            'url_template': f'https://www.trustpilot.com/review/www.{company_name}.com?page=',
            'selector': ('p', {'attrs': {'data-service-review-text-typography': 'true'}}),
        }
    }

    # Process each requested source
    for source in sources:
        if source not in source_configs:
            print(f"Source '{source}' is not supported.")
            continue

        config = source_configs[source]
        url_template = config['url_template']
        selector_type, selector_attrs = config['selector']

        # Function to scrape a single page
        def scrape_page(url):
            """Scrape reviews from a single page."""
            response = requests.get(url, headers=headers)

            if response.status_code == 200:
                soup = BeautifulSoup(response.content, 'html.parser')
                if selector_type == 'p' and 'attrs' in selector_attrs:
                    reviews = soup.find_all(selector_type, **selector_attrs['attrs'])
                else:
                    reviews = soup.find_all(selector_type, **selector_attrs)
                return [review.get_text() for review in reviews]
            else:
                print(f"Error scraping {url}. Status code: {response.status_code}")
                return []

        # Try to scrape first to know if the company exists
        url = url_template + str(1)
        response = requests.get(url, headers=headers)
        if response.status_code != 200:
            print(f"Company '{company_name}' not found on {source}.")
            continue

        # Scrape all requested pages
        source_reviews = []
        for page in range(1, num_pages + 1):
            url = url_template + str(page)
            print(f"Scraping {source} page {page}...")
            reviews = scrape_page(url)
            if not reviews:
                break  # Stop if no reviews are found on a page
            source_reviews.extend(reviews)
            time.sleep(delay)

        # Add source reviews to combined list
        all_reviews.extend(source_reviews)
        print(f"Added {len(source_reviews)} reviews from {source}")

    return all_reviews

In [ ]:
company = 'aetna'

reviews = scrape_reviews(company, sources=['bestcompany', 'trustpilot'], num_pages=3)

Scraping bestcompany page 1...
Scraping bestcompany page 2...
Scraping bestcompany page 3...
Added 150 reviews from bestcompany
Scraping trustpilot page 1...
Scraping trustpilot page 2...
Scraping trustpilot page 3...
Added 58 reviews from trustpilot


In [ ]:
for i in range(len(reviews)):
  print(i, "--------", reviews[i])

0 -------- I’m now age 77 and have Aetna. In 2015 I had open heart surgery at Doylestown Heart Hospital. I only had a small deductible and Aetna took over, and they were outstanding. Not one complaint.  In 2016 my wife had knee replacement, and again we were truly amazed as they picked up everything with small deductible. Recently had gall bladder surgery at Doylestown Hospital.  As usual, Aetna just worked as smooth as glass.  Amazing, a few days after leaving hospital, Aetna provided us with 14 free meals as part of our coverage. Wow! Aetna pays my fees at local YMCA with silver sneakers. We have plans C which INCLUDES DRUG coverage. Most of our prescriptions have NO co pay. We pay $72 each monthly. At my age it is hard to say WOW, but Aetna has performed better than could be expected. Luv Aetna. 👍🇱🇷
1 -------- Aetna is the absolute worst insurance company out there.  Been with Aetna for 12 years. I was diagnosed with Cancer in sept 2023 my surgery, radiation and continues treatment 

In [ ]:
def loading_models():
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  embedding_tokenizer = AutoTokenizer.from_pretrained('jinaai/jina-embeddings-v3')
  embedding_model = AutoModel.from_pretrained('jinaai/jina-embeddings-v3', trust_remote_code=True).to(device)